In [1]:
import time 

import numpy as np
from matplotlib import pyplot as plt
from difference_matrix import Difference_Matrix

### Styles of Recursion

Tail and Head Recursion

In [2]:
def sherman_morrison_recursion(a_ij,DDT_inv):
    """Compute the inverse of a matrix using the Sherman-Morrison formula
    
        Utilizes forward recrusion for computational efficiency
    """
    start=time.time()
    A_inv=DDT_inv
    k=1
    
    # Loop over the columns of the matrix
    while k<=len(a_ij):

        # Create the vectors u and v which are scaled unit vectors
        e_n=np.zeros(len(a_ij))
        e_n[k-1]=1
        u=e_n.reshape(-1,1)
        v=e_n.reshape(1,-1)
     
        num=A_inv.dot(a_ij[k-1]*u.dot(v)).dot(A_inv)
        den=1+a_ij[k-1]*v.dot(A_inv).dot(u)
     
        A_inv=A_inv -num/den
   
        k=k+1
    end=time.time()
    total_time=end-start
    return A_inv,total_time

In [3]:
def test_sherman_morrison(n):
    diff=Difference_Matrix(n,2)

    DDT=diff.DDT
    DDT_inv=diff.DDT_inv

    a_ij=np.random.rand(DDT.shape[0])

    # compute the inverse of DDT+a_ij*I using the Sherman-Morrison formula
    A_inv,total_time=sherman_morrison_recursion(a_ij,DDT_inv)

    # check that the inverse is correct
    computed_inv=A_inv.dot(DDT+np.diag(a_ij))

    print(f"Max error: {np.max(np.abs(computed_inv-np.eye(DDT.shape[0])))}")
    print(f"Total time: {total_time}")

    return
    




### 10 by 10 System

In [4]:
test_sherman_morrison(10)

Max error: 7.258083023486961e-15
Total time: 0.15099668502807617


### 100 by 100 System

In [5]:
test_sherman_morrison(100)

Max error: 4.454478452639736e-11
Total time: 0.17363262176513672


### 1000 x 1000 system; poor scaling

In [6]:
test_sherman_morrison(1000)

Max error: 8.212013346064754e-08
Total time: 109.09471130371094


### Improvement via Woodburry Matrix Identity (Blocked Sherman Morrison)

In [7]:
def woodbury_matrix_inversion(a_ij,DDT_inv,step=20):
    """Compute the inverse of a matrix using the Woodbury formula
    
       k-blocks of the matrix are inverted at a time
    """
    start=time.time()
    A_inv=DDT_inv
    k=1
    
    # Loop over the columns of the matrix
    while k<=len(a_ij):


        len_block=min(len(a_ij[k-1:k-1+step]),step)

        # Create the vectors u and v which are scaled unit vectors
        u=np.zeros((len(a_ij),len_block))
        v=np.zeros((len_block,len(a_ij)))
        c=np.zeros((len_block,len_block))

     

        for i in range(0,len_block):
         
            u[k+i-1,i]=1
            v[i,k+i-1]=1
            c[i,i]=a_ij[k+i-1]
        
        #  extract kth block of A_inv
        truncated_mat=v.dot(A_inv).dot(u)

        # compute the inverse of the kth block of A_inv
        inv_truncated_mat=np.linalg.inv(truncated_mat)

        # check that the inverse using numpy is correct
        assert np.max(np.abs(truncated_mat.dot(inv_truncated_mat)-np.eye(len_block)))<1e-10
        
        c_a_inv,tot_time=sherman_morrison_recursion(1/a_ij[k-1:k-1+step],inv_truncated_mat)

   
     
        A_inv=A_inv -A_inv.dot(u).dot(c_a_inv.dot(v).dot(A_inv))
   
        k=k+step
    end=time.time()
    total_time=end-start
    return A_inv,total_time

In [8]:
def test_woodbury_inversion(n,step=None):

    diff=Difference_Matrix(n,2)

    DDT=diff.DDT
    DDT_inv=diff.DDT_inv

    a_ij=np.random.rand(DDT.shape[0])

    if step is None:
        # compute the inverse of DDT+a_ij*I using the Woodbury formula
        A_inv,total_time=woodbury_matrix_inversion(a_ij,DDT_inv)
    else:
        # compute the inverse of DDT+a_ij*I using the Woodbury formula
        A_inv,total_time=woodbury_matrix_inversion(a_ij,DDT_inv,step=step)


    # check that the inverse is correct
    computed_inv=A_inv.dot(DDT+np.diag(a_ij))

    print(f"Max error: {np.max(np.abs(computed_inv-np.eye(DDT.shape[0])))}")
    print(f"Total time: {total_time}")

    return

In [9]:
test_woodbury_inversion(100)

Max error: 1.8761702087755072e-09
Total time: 0.014874696731567383


In [10]:
test_woodbury_inversion(1000)

Max error: 8.289322674422576e-08
Total time: 0.938239336013794


In [13]:
test_woodbury_inversion(2500)


Max error: 1.0577326039550462e-06
Total time: 12.997475385665894
